In [11]:
# -*- coding: utf-8 -*-
'''An implementation of sequence to sequence learning for performing addition
Input: "535+61"
Output: "596"
Padding is handled by using a repeated sentinel character (space)
Input may optionally be inverted, shown to increase performance in many tasks in:
"Learning to Execute"
http://arxiv.org/abs/1410.4615
and
"Sequence to Sequence Learning with Neural Networks"
http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
Theoretically it introduces shorter term dependencies between source and target.
Two digits inverted:
+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs
Three digits inverted:
+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs
Four digits inverted:
+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs
Five digits inverted:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs
'''

from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range


class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.
        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

# Parameters for the model and dataset.
TRAINING_SIZE = 150000
DIGITS = 3
INVERT = False

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789* '
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}*{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a * b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS * 2 - len(ans))
    #if INVERT:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        # query = query[::-1]
        
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))
print(questions[:10], expected[:10])

print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS * 2, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS* 2)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last hidden state of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS * 2))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if INVERT else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

Generating data...
Total addition questions: 150000
['77*9   ', '95*410 ', '62*2   ', '258*5  ', '15*908 ', '5*35   ', '80*795 ', '489*0  ', '563*918', '4*54   '] ['693   ', '38950 ', '124   ', '1290  ', '13620 ', '175   ', '63600 ', '0     ', '516834', '216   ']
Vectorization...
Training Data:
(135000, 7, 12)
(135000, 6, 12)
Validation Data:
(15000, 7, 12)
(15000, 6, 12)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_17 (LSTM)               (None, 128)               72192     
_________________________________________________________________
repeat_vector_9 (RepeatVecto (None, 6, 128)            0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 6, 128)            131584    
_________________________________________________________________
time_distributed_9 (TimeDist (None, 6, 12)             1548      
________________

135000/135000 [==============================] - 19s 144us/step - loss: 1.0762 - acc: 0.5564 - val_loss: 1.0594 - val_acc: 0.5643
Q 283*379 T 107257 ☒ 104443
Q 540*133 T 71820  ☒ 72240 
Q 224*250 T 56000  ☒ 53000 
Q 74*233  T 17242  ☒ 17744 
Q 53*934  T 49502  ☒ 49444 
Q 976*119 T 116144 ☒ 116444
Q 729*14  T 10206  ☒ 10124 
Q 3*572   T 1716   ☒ 1788  
Q 74*139  T 10286  ☒ 10182 
Q 550*215 T 118250 ☒ 113750

--------------------------------------------------
Iteration 13
Train on 135000 samples, validate on 15000 samples
Epoch 1/1
135000/135000 [==============================] - 19s 144us/step - loss: 1.0681 - acc: 0.5585 - val_loss: 1.0544 - val_acc: 0.5588
Q 365*543 T 198195 ☒ 108385
Q 165*978 T 161370 ☒ 156490
Q 284*552 T 156768 ☒ 155584
Q 75*84   T 6300   ☒ 6250  
Q 742*315 T 233730 ☒ 230330
Q 512*32  T 16384  ☒ 16758 
Q 588*94  T 55272  ☒ 54334 
Q 705*32  T 22560  ☒ 22030 
Q 58*613  T 35554  ☒ 35244 
Q 272*388 T 105536 ☒ 105554

--------------------------------------------------
It

135000/135000 [==============================] - 20s 145us/step - loss: 0.7992 - acc: 0.6887 - val_loss: 0.7855 - val_acc: 0.6912
Q 320*927 T 296640 ☒ 290840
Q 82*328  T 26896  ☒ 26216 
Q 902*96  T 86592  ☒ 86672 
Q 382*618 T 236076 ☒ 232076
Q 745*56  T 41720  ☒ 41020 
Q 30*424  T 12720  ☑ 12720 
Q 97*386  T 37442  ☒ 37562 
Q 277*719 T 199163 ☒ 198083
Q 679*722 T 490238 ☒ 486858
Q 530*131 T 69430  ☒ 66730 

--------------------------------------------------
Iteration 26
Train on 135000 samples, validate on 15000 samples
Epoch 1/1
135000/135000 [==============================] - 19s 143us/step - loss: 0.7886 - acc: 0.6906 - val_loss: 0.7692 - val_acc: 0.6973
Q 365*303 T 110595 ☒ 110335
Q 15*349  T 5235   ☒ 5335  
Q 672*586 T 393792 ☒ 398892
Q 16*146  T 2336   ☒ 2276  
Q 30*832  T 24960  ☒ 25360 
Q 309*314 T 97026  ☒ 97986 
Q 99*738  T 73062  ☒ 73382 
Q 245*67  T 16415  ☒ 16305 
Q 695*503 T 349585 ☒ 349135
Q 797*160 T 127520 ☒ 126720

--------------------------------------------------
It

135000/135000 [==============================] - 19s 143us/step - loss: 0.6856 - acc: 0.7209 - val_loss: 0.6908 - val_acc: 0.7138
Q 873*6   T 5238   ☑ 5238  
Q 597*88  T 52536  ☒ 52016 
Q 44*404  T 17776  ☒ 17616 
Q 463*92  T 42596  ☒ 42796 
Q 21*251  T 5271   ☒ 5291  
Q 996*3   T 2988   ☒ 2968  
Q 50*737  T 36850  ☑ 36850 
Q 432*250 T 108000 ☒ 107000
Q 29*74   T 2146   ☒ 2126  
Q 171*532 T 90972  ☒ 91532 

--------------------------------------------------
Iteration 39
Train on 135000 samples, validate on 15000 samples
Epoch 1/1
135000/135000 [==============================] - 19s 144us/step - loss: 0.6798 - acc: 0.7227 - val_loss: 0.6721 - val_acc: 0.7222
Q 416*735 T 305760 ☒ 302260
Q 81*491  T 39771  ☒ 39241 
Q 480*686 T 329280 ☒ 322480
Q 173*22  T 3806   ☒ 3786  
Q 158*75  T 11850  ☒ 11950 
Q 98*245  T 24010  ☒ 23670 
Q 0*131   T 0      ☑ 0     
Q 322*117 T 37674  ☒ 37974 
Q 349*659 T 229991 ☒ 228851
Q 54*148  T 7992   ☒ 7972  

--------------------------------------------------
It

135000/135000 [==============================] - 19s 144us/step - loss: 0.6360 - acc: 0.7397 - val_loss: 0.6577 - val_acc: 0.7242
Q 111*524 T 58164  ☒ 58944 
Q 380*71  T 26980  ☒ 27180 
Q 99*234  T 23166  ☒ 23066 
Q 348*279 T 97092  ☒ 97592 
Q 652*824 T 537248 ☒ 530268
Q 1*14    T 14     ☑ 14    
Q 139*86  T 11954  ☑ 11954 
Q 875*22  T 19250  ☒ 19350 
Q 752*99  T 74448  ☒ 73928 
Q 215*264 T 56760  ☒ 56860 

--------------------------------------------------
Iteration 52
Train on 135000 samples, validate on 15000 samples
Epoch 1/1
135000/135000 [==============================] - 19s 143us/step - loss: 0.6312 - acc: 0.7416 - val_loss: 0.6348 - val_acc: 0.7363
Q 563*708 T 398604 ☒ 498584
Q 347*63  T 21861  ☒ 21981 
Q 30*191  T 5730   ☑ 5730  
Q 525*71  T 37275  ☒ 37725 
Q 796*450 T 358200 ☑ 358200
Q 684*137 T 93708  ☒ 94248 
Q 630*173 T 108990 ☒ 109190
Q 373*972 T 362556 ☒ 364956
Q 875*718 T 628250 ☒ 632350
Q 379*27  T 10233  ☒ 10243 

--------------------------------------------------
It

...

135000/135000 [==============================] - 19s 142us/step - loss: 0.4768 - acc: 0.8017 - val_loss: 0.5371 - val_acc: 0.7788
Q 851*308 T 262108 ☒ 262408
Q 44*894  T 39336  ☒ 39396 
Q 73*818  T 59714  ☒ 59794 
Q 996*17  T 16932  ☒ 16972 
Q 915*172 T 157380 ☒ 157680
Q 962*313 T 301106 ☒ 300226
Q 467*789 T 368463 ☒ 368883
Q 735*212 T 155820 ☒ 155520
Q 189*939 T 177471 ☒ 178891
Q 29*32   T 928    ☑ 928   

--------------------------------------------------
Iteration 143
Train on 135000 samples, validate on 15000 samples
Epoch 1/1
135000/135000 [==============================] - 19s 142us/step - loss: 0.4743 - acc: 0.8023 - val_loss: 0.4943 - val_acc: 0.7905
Q 124*781 T 96844  ☒ 96644 
Q 834*782 T 652188 ☒ 652768
Q 600*172 T 103200 ☑ 103200
Q 849*57  T 48393  ☒ 48533 
Q 45*276  T 12420  ☑ 12420 
Q 348*999 T 347652 ☒ 347112
Q 883*18  T 15894  ☒ 15874 
Q 656*746 T 489376 ☒ 490656
Q 752*99  T 74448  ☒ 74768 
Q 976*219 T 213744 ☒ 214684

--------------------------------------------------
I

135000/135000 [==============================] - 20s 147us/step - loss: 0.4594 - acc: 0.8092 - val_loss: 0.5006 - val_acc: 0.7860
Q 4*394   T 1576   ☑ 1576  
Q 368*39  T 14352  ☑ 14352 
Q 475*767 T 364325 ☒ 365625
Q 8*611   T 4888   ☑ 4888  
Q 500*378 T 189000 ☑ 189000
Q 379*562 T 212998 ☒ 213238
Q 508*16  T 8128   ☒ 8068  
Q 903*9   T 8127   ☑ 8127  
Q 6*516   T 3096   ☑ 3096  
Q 950*45  T 42750  ☒ 42650 

--------------------------------------------------
Iteration 156
Train on 135000 samples, validate on 15000 samples
Epoch 1/1
135000/135000 [==============================] - 19s 143us/step - loss: 0.4678 - acc: 0.8062 - val_loss: 0.5164 - val_acc: 0.7820
Q 744*27  T 20088  ☒ 20108 
Q 321*89  T 28569  ☒ 28449 
Q 20*503  T 10060  ☑ 10060 
Q 34*667  T 22678  ☒ 22638 
Q 953*577 T 549881 ☒ 540181
Q 654*50  T 32700  ☑ 32700 
Q 4*698   T 2792   ☑ 2792  
Q 325*311 T 101075 ☒ 100875
Q 157*57  T 8949   ☒ 8909  
Q 86*782  T 67252  ☒ 67192 

--------------------------------------------------
I

135000/135000 [==============================] - 19s 143us/step - loss: 0.4531 - acc: 0.8123 - val_loss: 0.5011 - val_acc: 0.7855
Q 124*146 T 18104  ☒ 17724 
Q 89*784  T 69776  ☒ 69736 
Q 845*260 T 219700 ☒ 220700
Q 769*961 T 739009 ☒ 743169
Q 11*924  T 10164  ☒ 10124 
Q 156*874 T 136344 ☒ 135584
Q 445*31  T 13795  ☒ 13895 
Q 675*99  T 66825  ☑ 66825 
Q 585*225 T 131625 ☒ 131125
Q 454*35  T 15890  ☒ 15990 

--------------------------------------------------
Iteration 169
Train on 135000 samples, validate on 15000 samples
Epoch 1/1
135000/135000 [==============================] - 19s 144us/step - loss: 0.4553 - acc: 0.8114 - val_loss: 0.5296 - val_acc: 0.7759
Q 781*584 T 456104 ☒ 454904
Q 9*721   T 6489   ☑ 6489  
Q 10*655  T 6550   ☑ 6550  
Q 211*266 T 56126  ☒ 56186 
Q 718*880 T 631840 ☒ 630640
Q 588*446 T 262248 ☒ 261448
Q 429*47  T 20163  ☒ 20223 
Q 220*784 T 172480 ☑ 172480
Q 39*683  T 26637  ☒ 26697 
Q 546*99  T 54054  ☒ 54254 

--------------------------------------------------
I

135000/135000 [==============================] - 19s 142us/step - loss: 0.4469 - acc: 0.8153 - val_loss: 0.4990 - val_acc: 0.7916
Q 89*213  T 18957  ☒ 18937 
Q 93*371  T 34503  ☒ 34403 
Q 935*522 T 488070 ☒ 488270
Q 234*8   T 1872   ☑ 1872  
Q 751*586 T 440086 ☒ 438086
Q 609*24  T 14616  ☒ 14656 
Q 694*56  T 38864  ☒ 38884 
Q 31*530  T 16430  ☑ 16430 
Q 69*373  T 25737  ☒ 25717 
Q 20*958  T 19160  ☑ 19160 

--------------------------------------------------
Iteration 182
Train on 135000 samples, validate on 15000 samples
Epoch 1/1
135000/135000 [==============================] - 19s 142us/step - loss: 0.4487 - acc: 0.8143 - val_loss: 0.5035 - val_acc: 0.7855
Q 645*1   T 645    ☑ 645   
Q 353*317 T 111901 ☒ 111321
Q 13*141  T 1833   ☒ 1893  
Q 372*180 T 66960  ☒ 66160 
Q 35*451  T 15785  ☒ 15885 
Q 43*100  T 4300   ☑ 4300  
Q 64*148  T 9472   ☒ 9412  
Q 30*786  T 23580  ☑ 23580 
Q 163*32  T 5216   ☒ 5136  
Q 491*67  T 32897  ☒ 32837 

--------------------------------------------------
I

135000/135000 [==============================] - 19s 143us/step - loss: 0.4451 - acc: 0.8161 - val_loss: 0.5054 - val_acc: 0.7883
Q 442*232 T 102544 ☒ 102724
Q 12*204  T 2448   ☑ 2448  
Q 803*650 T 521950 ☑ 521950
Q 12*53   T 636    ☑ 636   
Q 348*419 T 145812 ☑ 145812
Q 902*270 T 243540 ☒ 242140
Q 486*46  T 22356  ☒ 22276 
Q 21*935  T 19635  ☒ 19535 
Q 19*811  T 15409  ☒ 15309 
Q 55*459  T 25245  ☑ 25245 

--------------------------------------------------
Iteration 195
Train on 135000 samples, validate on 15000 samples
Epoch 1/1
135000/135000 [==============================] - 19s 144us/step - loss: 0.4507 - acc: 0.8144 - val_loss: 0.4846 - val_acc: 0.7971
Q 666*953 T 634698 ☒ 633078
Q 93*216  T 20088  ☑ 20088 
Q 629*670 T 421430 ☒ 420830
Q 659*52  T 34268  ☒ 34288 
Q 583*561 T 327063 ☒ 327343
Q 976*795 T 775920 ☒ 774220
Q 183*64  T 11712  ☒ 11792 
Q 199*69  T 13731  ☒ 13711 
Q 863*812 T 700756 ☒ 700036
Q 968*773 T 748264 ☒ 746964

--------------------------------------------------
I